In [ ]:
# Cloning the project repository
! git clone git@github.com:yupopov/fantlab-recommender-system.git 
import os
os.chdir('fantlab-recommender-system')

Cloning into 'fantlab-recommender-system'...
remote: Enumerating objects: 578, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 578 (delta 112), reused 112 (delta 53), pack-reused 403
Receiving objects: 100% (578/578), 545.34 MiB | 24.63 MiB/s, done.
Resolving deltas: 100% (307/307), done.


In [ ]:
! git add README.md
! git commit -m 'update README.md'
! git push origin main

[main 213e2e2] update README.md
 1 file changed, 6 insertions(+), 6 deletions(-)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 369 bytes | 369.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:yupopov/fantlab-recommender-system.git
   8688a60..213e2e2  main -> main


In [ ]:
%%capture
% load_ext autoreload
% autoreload 2

import sys
import gzip
from string import punctuation
import requests
import json
import re
import time
from collections import defaultdict, Counter
from tqdm.auto import tqdm
import pickle
from pprint import pprint

import numpy as np
import pandas as pd
import torch
from scipy.sparse import csr_matrix, csc_matrix, save_npz, load_npz
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
! pip install nltk
! pip install gensim==4.2.0
! pip install pymorphy2
! pip install aiohttp
! pip install transformers sentencepiece
! pip install lightfm
import lightfm
from transformers import AutoTokenizer, AutoModel
from nltk import word_tokenize
import gensim
import pymorphy2
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from string import punctuation
from scipy.sparse import save_npz, load_npz


from src.data_retrieval.html_extraction import get_work_ids
from src.preprocessing.title_parser import TitleParser
from src.preprocessing.item_features_buildup import item_features_buildup
from src.models.LinearRecommender import LinearRecommender
from src.models.get_top_k_predictions_with_label import get_top_k_predictions_with_labels

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Data preprocessing

In [ ]:
# Load and parse informations about books
with gzip.open('./data/raw/work_infos.json.gz', 'rt') as f:
    work_infos = json.load(f)

title_parser = TitleParser(normalize_description=True)
works_df = title_parser(work_infos)

works_df.head()

  0%|          | 0/23867 [00:00<?, ?it/s]

,work_name,work_description,work_name_bonus,work_notes,authors,lang,genre_infos,genre_group_1_labels,genre_group_2_labels,genre_group_3_labels,genre_group_4_labels,genre_group_6_labels,genre_group_7_labels,genre_group_5_labels,awards
work_id,,,,,,,,,,,,,,,
103954,Сотый сон Кэрол Онейр,кэрол онейра сочинять сон простой золотой сон-...,,,диана_уинн_джонс,английский,"[(19, Фэнтези, 7, 0.9), (32, Сказка/Притча, 5,...",фэнтези сказка/притча,приключенческое психологическое,параллельный_мир/вселенная,неопределенное_время_действия,становление/взросление_героя,линейный,любой детская_литература,
234281,На прорыв времени! Российский спецназ против г...,способный бригада российский спецназ перенести...,,,сергей_артюхин,русский,"[(1, Фантастика, 6, 1), (64, Военное, 6, 1), (...",фантастика,военное,наш_мир_(земля) альтернативная_история_нашего_...,20_век,путешественники_(попаданцы) прогрессорство,линейно-параллельный,любой,
268642,Фреон,хмурый вечно замкнутый сталкер-одиночка фреон ...,,книга являться непрямой продолжение роман лунь,сергей_клочков,русский,"[(1, Фантастика, 5, 1), (52, Приключенческое, ...",фантастика,приключенческое на_основе_игровой_вселенной,наш_мир_(земля),21_век,путешествие_к_особой_цели,линейный_с_экскурсами,любой,
174314,Неосквернённый,голова полоумный волк жить демон который спаса...,,,грегори_киз,английский,"[(19, Фэнтези, 40, 1), (52, Приключенческое, 3...",фэнтези,приключенческое,"другой_мир,_не_связанный_с_нашим",неопределенное_время_действия,путешествие_к_особой_цели,линейный,только_для_взрослых,
34830,Железный сокол Гардарики,среди сотрудник институт экспериментальный ист...,,,владимир_свержин,русский,"[(1, Фантастика, 11, 1), (52, Приключенческое,...",фантастика,приключенческое ироническое,альтернативная_история_нашего_мира_(земли),эпоха_географических_открытий_(15-16_века),путешественники_(попаданцы) путешествие_к_особ...,линейный,любой,


In [ ]:
# Load marks infos for all the works
marks_df = pd.read_csv('data/raw/work_marks.csv.gz', parse_dates=['date'])
marks_df.head()

,user_id,work_id,mark,date
0,48027,103954,10,2010-08-18
1,61504,103954,10,2017-07-15
2,192139,103954,10,2021-06-10
3,118819,103954,10,2014-04-22
4,71960,103954,10,2012-04-19


# Recommendations
 

In [ ]:
# recommends by k nearest neighbours
def recommend_by_knn(work_name, embeddings, key2index: dict, works_df=works_df):
    key = works_df.query(f'work_name == "{work_name}"').index.tolist()[0]
    index = key2index[f'{key}']
    vector = embeddings[int(index)]
    if vector.shape[0] != embeddings.shape[1]:
        raise ValueError('The vector and the embeddings have different dimensions!')
    dist = (embeddings - vector).norm(dim=1)
    knn = dist.topk(10, largest=False)
    top_k_indices = knn.indices
    return works_df.iloc[top_k_indices]['work_name']

In [ ]:
# load normalized descriptions for linear models 
with gzip.open('./data/interim/work_descriptions_normalized.json.gz', 'rt') as f:
    work_descriptions_normalized = json.load(f)

In [ ]:
# load unnormalized descriptions for rubert-tiny
with gzip.open('./data/interim/work_descriptions_unnormalized.json.gz', 'rt') as f:
    work_descriptions_unnormalized = json.load(f)

## Linear recommender

In [ ]:
tfidf_embs = load_npz('./data/interim/work_tfidf_matrix.npz')

In [ ]:
with gzip.open('./data/interim/key2index.json.gz', 'rt') as f:
    key2index = json.load(f)

In [ ]:
with open('./data/interim/LightFmDataset.pkl', 'rb') as f:
   fm_dataset = pickle.load(f)

In [ ]:
lr = LinearRecommender(fm_dataset, tfidf_embs, key2index)
preds, labels = get_top_k_predictions_with_labels(lr, fm_dataset.test_data, fm_dataset.train_data)

  0%|          | 0/212 [00:00<?, ?it/s]

In [ ]:
works_df.iloc[preds[1234]].work_name

work_id
84840                  Иные песни
13092                     Зеркала
35734             Год нашей войны
76569                400 поганцев
1079                    Баскетбол
37266     Пространство Откровения
176712                     Вампир
349375           Народ обреченный
479111               Единственная
480301     Наперегонки со смертью
Name: work_name, dtype: object

# tfidf

In [ ]:
tfidf = TfidfVectorizer(min_df=10)

In [ ]:
# recommend function for tfidf using cosine similarity to compare work descriptions 
# recommends 10 most similar to the chosen one
def recommend(work_name, work_descriptions=work_descriptions_normalized,
              df=works_df, vectorizer=tfidf):
    work_descriptions = [value for key, value in work_descriptions.items()]
    recommended_works = []
    title_text_features = vectorizer.fit_transform(work_descriptions)
    # normalize the features row-by-row to obtain cosine similarities
    title_text_features = normalize(title_text_features, norm='l2', axis=1)

   
    idx = works_df.index.get_loc(works_df.index[works_df['work_name']==work_name][0])
    score_series = (title_text_features @ title_text_features[idx].T).toarray()
    score_series = pd.Series(np.array([elem[0] for elem in score_series])).sort_values(ascending=False)
    top_10_indices = list(score_series.iloc[1:21].index)
    recommended_works = df.iloc[top_10_indices].work_name
    
    return recommended_works

As we can see, TF-IDF recommends pretty similar works to the given one

as far as it can give out similar ones at all

In [ ]:
recommend('Ведьмак')

work_id
3054       Что-то кончается, что-то начинается
2985                               Меньшее Зло
2973                         Предел возможного
10051                                   Нянька
13858                                  No Past
2978                             Нечто большее
2959                           Ведьмак Геральт
10039                       Родина безразличия
118003                         Ведьмачье слово
132233                    Поезд вне расписания
491411                              Сезон гроз
2986                               Вопрос цены
43027                                  Матадор
2987                                Край света
96915                            Осколок жизни
1392537                     Я спас СССР! Том V
3231                                    Второй
109371             Наполним чаши телескопов...
561                        Сказка о Тройке — 1
2975                              Вечный огонь
Name: work_name, dtype: object

In [ ]:
recommend('Две твердыни')

work_id
1669                 Возвращение Короля
3328                   Хроники Хоукмуна
79                              Всадник
1667                    Братство Кольца
436546                    Отцова забота
2009      Шкатулка, в которой была Тьма
26000               Без пятнадцати семь
77051             Возвращение в Форност
8150                  Сказочный корабль
4730                Келльская пророчица
361322            Клинок его Величества
59242               Эрагон. Возвращение
116055              Колдовская компания
1666         Хоббит, или Туда и обратно
3746                         Игра теней
81378                  Девятая крепость
6880            Поведай мне свои печали
164350               Серебряный ошейник
201142              Метро 2033: К свету
65630                   Путь разрушения
Name: work_name, dtype: object

In [ ]:
recommend('1984')

work_id
157581                        Замкнутый круг
31535                     О дивный новый мир
41167                              Перешейцы
14268                             В Дономаге
5346                          Духовное ружье
20222                 Мир, в котором я исчез
100061    Если бы я проснулась перед смертью
46093                             Сад Адомфы
14805                               Имортист
8146                 Мир одного дня: Бунтарь
23323              «Мне ещё ехать далеко...»
235130                      Руигат. Рождение
31541                                 Остров
81862                              Два врача
18622              Кукла, которая делает всё
198896                               Зародыш
74306                                   Стая
12934                           Шесть гениев
523631                                      
7609           Бледный всадник, Чёрный Валет
Name: work_name, dtype: object

# rubert-tiny

In [ ]:
# Load pretrained distilled BERT for russian language
# see https://habr.com/ru/post/562064/
bert_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
bert_model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
bert_model.cuda()

In [ ]:
# returns embeddings of unnormalized descriptions
def embed_work_text_features_bert(texts, tokenizer=bert_tokenizer, model=bert_model, batch_size=40):
    embeddings = []
    texts = [value for key, value in texts.items()]
    for batch_idx in tqdm(range(0, len(texts), batch_size)):
        batch = texts[batch_idx: batch_idx + batch_size]
        tokenized_batch = tokenizer(batch, padding='max_length', truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**{key: value.to(model.device) for key, value in tokenized_batch.items()})
        model_embeddings = model_output.last_hidden_state[:, 0, :]
        model_embeddings = torch.nn.functional.normalize(model_embeddings)
        embeddings.append(model_embeddings.cpu().numpy())
    # return np.array(embeddings)
    return np.concatenate(embeddings)

In [ ]:
bert_embeddings = embed_work_text_features_bert(work_descriptions_unnormalized, batch_size=20)

  0%|          | 0/1194 [00:00<?, ?it/s]

In [ ]:
bert_embeddings = torch.tensor(bert_embeddings)

In [ ]:
with gzip.open('./data/interim/key2index.json.gz', 'rt') as f:
    key2index = json.load(f)

Here little BERT shows rather poor results

In [ ]:
recommend_by_knn('Ведьмак', bert_embeddings, key2index)

tensor([0.0000, 0.2993, 0.3035, 0.3069, 0.3086])


work_id
2983                                         Ведьмак
192058                            Магическая страсть
77678                                          Дуэль
46556                                Арканарский вор
596017                                 Петрушкин Лог
166491                                   Дети Ашшура
1343                                         Дачники
259260                                       Туннели
146292                           Ниал из Земли Ветра
680262    Хозяйка Серых земель. Капкан на волкодлака
Name: work_name, dtype: object

# Navec


In [ ]:
! wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
! pip install navec
! pip install navec slovnet
from navec import Navec
from slovnet.model.emb import NavecEmbedding

In [ ]:
navec = Navec.load('./navec_hudlit_v1_12B_500K_300d_100q.tar')
emb = NavecEmbedding(navec)

In [ ]:
def embed_with_navec(embs, texts):
    tokenized_texts = []
    for text in tqdm(texts):
      token_ids = [navec.vocab.get(word, navec.vocab.unk_id) for word in text]
      tokenized_texts.append(token_ids)
    tokenized_texts = [text[:120] for text in tokenized_texts]
    pad_ind = navec.vocab.get('<pad>')
    tokenized_texts = np.array([np.pad(text, (0, 120 - len(text)), 'constant', constant_values=[pad_ind]) for text in tokenized_texts])
    tokenized_texts = torch.tensor(tokenized_texts)
    embeddings = [embs(tokenized_text).mean(axis=0).cpu().numpy() for tokenized_text in tqdm(tokenized_texts)]
    return embeddings

In [ ]:
navec_embeddings = embed_with_navec(emb, work_descriptions_normalized)

  0%|          | 0/23867 [00:00<?, ?it/s]

  0%|          | 0/23867 [00:00<?, ?it/s]

In [ ]:
navec_embeddings = torch.tensor(navec_embeddings)

In [ ]:
with gzip.open('./data/interim/key2index.json.gz', 'rt') as f:
    key2index = json.load(f)

It seems that navec pretrained embeddings work better than BERT

but they are still much worse than TF-IDF

In [ ]:
recommend_by_knn('Ведьмак', navec_embeddings, key2index)

tensor([0., 0., 0., 0., 0.])


work_id
6011       За железной дверью
4023              Меч неверия
7293            Давние потери
1971                         
3590              Принц крови
3962            Сказительница
4774       Глина господа бога
1782        Повелитель кольца
7939    Банка фруктового сока
7862       Господа альбатросы
Name: work_name, dtype: object

# Doc2Vec

In [ ]:
# Load item categorical features obtained from works_df with 
# item_featires_buildup.py
with gzip.open('./data/raw/item_features.json.gz', 'rt') as f:
    item_features = json.load(f)

In [ ]:
# tokenize descriptions
work_descriptions_tokenized = {k: v.split() for k, v in work_descriptions_normalized.items()}

In [ ]:
# make item features suitable for use in doc2vec.TaggedDocument
from collections import defaultdict
work_tags = defaultdict(list)
for item in item_features:
    item_id = item[0]
    tag_name = item[2]
    work_tags[item_id].append(tag_name)

In [ ]:
# make the dstasets for Doc2Vec 
train_data = [gensim.models.doc2vec.TaggedDocument(value, [key]) for key, value in work_descriptions_tokenized.items()]
train_data_tagged = [gensim.models.doc2vec.TaggedDocument(value, [key] + work_tags[key]) for key, value in work_descriptions_tokenized.items()]

In [ ]:
model_params = {'vector_size': 50, 'min_count':10, 
               'epochs':20, 'workers': 2} 


model = gensim.models.doc2vec.Doc2Vec(**model_params)
model.build_vocab(train_data, progress_per=10000)

model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

Doc2Vec also does not give us similar to given works, looks very strange

In [ ]:
work_vector = model.infer_vector(work_descriptions_tokenized['2965'], epochs=1000)
# model_.dv = model.__dict__['docvecs'] # need to use if gensim is downloaded by Colab's default 
                                        # ver 3.6.0 instead of actual 4.2.0
                                        
most_similar = model.dv.most_similar([work_vector], topn=len(model.dv))
most_similar_idx = [item_id for item_id, score in most_similar if item_id in work_descriptions_tokenized]
works_df.loc[most_similar_idx[:10]]

In [ ]:
def make_doc2vec_embs(data: dict, tags: dict = None,
                      model_params: dict={'vector_size': 120, 'min_count':10, 
                                          'epochs':1, 'workers': 2}, 
                      infer_params: dict={'epochs': 1}):

    if tags is not None:
        train_data = [gensim.models.doc2vec.TaggedDocument(data[key], tags[key]) for key in data.keys()]
    else:
        train_data = [gensim.models.doc2vec.TaggedDocument(value, [key]) for key, value in data.items()]

    print('Training model...\n')
    model = gensim.models.doc2vec.Doc2Vec(**model_params)
    # model.dv = model.__dict__['docvecs'] # declare a function which we cannot use by default
    model.build_vocab(train_data)
    model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

    print('Building embeddings...')
    embeddings = {key: model.infer_vector(doc, **infer_params) for key, doc in tqdm(data.items())}
    print(f'Embeddings for {len(embeddings)} works built')

    return embeddings

In [ ]:
d2v_embs = make_doc2vec_embs(work_descriptions_tokenized, work_tags)

# word2vec

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model__ = Word2Vec(work_descriptions_normalized, **model_params)

In [ ]:
def make_word2vec_embs(data: dict,
                      model_params: dict={'vector_size':0, 'window':5, 
                                          'min_count':10, 'workers': 2},
                      train_params={'epochs':40}):


    train_data = [value for key, value in data.items()]
    print('Training model...\n')
    model = gensim.models.Word2Vec(train_data, **model_params)
    model.train(train_data, total_examples=model.corpus_count, **train_params)

    print('Building embeddings...')
    embeddings = {key: [model.wv[word] for word in doc if word in model.wv] for key, doc in tqdm(data.items())}

    print(f'Embeddings for {len(embeddings)} works built')

    return embeddings


In [ ]:
w2v_embs = make_word2vec_embs(work_descriptions_tokenized)

2022-05-22 05:04:10,390 : INFO : collecting all words and their counts
2022-05-22 05:04:10,392 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-05-22 05:04:10,509 : INFO : PROGRESS: at sentence #10000, processed 366926 words, keeping 39523 word types


Training model...



2022-05-22 05:04:10,637 : INFO : PROGRESS: at sentence #20000, processed 736136 words, keeping 58426 word types
2022-05-22 05:04:10,695 : INFO : collected 64475 word types from a corpus of 879656 raw words and 23867 sentences
2022-05-22 05:04:10,696 : INFO : Creating a fresh vocabulary
2022-05-22 05:04:10,768 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 retains 9567 unique words (14.84% of original 64475, drops 54908)', 'datetime': '2022-05-22T05:04:10.768615', 'gensim': '4.2.0', 'python': '3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2022-05-22 05:04:10,770 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=10 leaves 765559 word corpus (87.03% of original 879656, drops 114097)', 'datetime': '2022-05-22T05:04:10.770349', 'gensim': '4.2.0', 'python': '3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.188+-x86_64-with-Ubuntu-18.0

Building embeddings...


  0%|          | 0/23867 [00:00<?, ?it/s]

Embeddings for 23867 works built
